In [ ]:
# Diseño UML y Estructura de Base de Datos
# =============================================================================
print("""
🏗️ DIAGRAMA UML - Estructura de la Base de Datos

┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│     AUTORES     │    │  LIBROS_AUTORES  │    │     LIBROS      │
├─────────────────┤    │   (Tabla Pivot)  │    ├─────────────────┤
│ id (PK)         │◄───┤ autor_id (FK)    │───►│ id (PK)         │
│ nombre          │    │ libro_id (FK)    │    │ titulo          │
└─────────────────┘    └──────────────────┘    │ precio          │
                                               │ rating          │
┌─────────────────┐                            │ categoria_id(FK)│
│   CATEGORIAS    │◄───────────────────────────│ en_stock        │
├─────────────────┤                            │ url             │
│ id (PK)         │                            └─────────────────┘
│ nombre          │
└─────────────────┘

Relaciones:
- AUTORES ↔ LIBROS: Muchos a Muchos (un autor puede escribir varios libros, 
  un libro puede tener varios autores)
- CATEGORIAS ↔ LIBROS: Uno a Muchos (una categoría tiene muchos libros)
""")

In [ ]:
# Creación de Base de Datos (DDL)
# =============================================================================
import psycopg2

conexionDB = psycopg2.connect(
    host = "localhost",
    port = "5432",
    database = "libros",
    user = "postgres",
    password = "dan"
)
print("conexion exitosa")

cursor = conexionDB.cursor() # el cursor es el medio por el que envia y recibe los comandos sql

# tabla de autores
cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores(
        id SERIAL PRIMARY KEY,
        nombre VARCHAR (100)
)
""")
# tabla de categorias
cursor.execute("""
    CREATE TABLE IF NOT EXISTS categorias(
        id SERIAL PRIMARY KEY,
        nombre VARCHAR (100)
)
""")
# tabla de libros
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros(
        id SERIAL PRIMARY KEY,
        titulo VARCHAR (500) NOT NULL,
        precio DECIMAL (10,2),
        rating INTEGER CHECK (rating >= 0 AND rating <= 5),
        categoria_id INTEGER,
        en_stock VARCHAR (100),
        url VARCHAR (1000),
        FOREIGN KEY (categoria_id) REFERENCES categorias (id)
        
)
""")
#titulo, precio, rating, categoria, en stock, fecha de publicacion, descripcion y url
# Tabla autores muchos <-> muchos
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros_autores(
        id SERIAL PRIMARY KEY,
        libro_id INTEGER,
        autor_id INTEGER,
        FOREIGN KEY (libro_id) REFERENCES libros(id) ON DELETE CASCADE,
        FOREIGN KEY (autor_id) REFERENCES autores(id) ON DELETE CASCADE,
        UNIQUE(libro_id, autor_id)
)
""")


conexionDB.commit()
print("Tabla creada exitosamente")

cursor.close()
conexionDB.close()

conexion exitosa
Tabla creada exitosamente


In [ ]:
# Funciones de Web Scraping
# =============================================================================
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import json

scrap = requests.session() # mantengo la session abierta
# lista de autores para asignar de manera aleatoria
autores = [
    "Juan Pérez",
    "María García",
    "Carlos López", 
    "Ana Martínez", 
    "Luis Rodríguez", 
    "Sofia Fernández"
]

# diccionario para covertir los str a numeros
reating_convercion = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

# funcion donde scrapeo la pagina books to scrape
def scrapeaTodo(i):
    paginas = scrap.get(f"https://books.toscrape.com/catalogue/page-{i}.html") # le doy como parametro i para luego recorrer las 50 paginas
    paginas.encoding = "utf-8" # codifico todos los caracteres para que scrapeo
    soup = BeautifulSoup(paginas.text,"html.parser") # parseo(agrega etiquetas html)
    
    datos_libros = [] # para guardar datos de los libros

    for lib in soup.find_all("article", class_= "product_pod"): # recorro todos los articulos con esa clase
        titulo = lib.h3.a["title"].strip()
        rating = reating_convercion[lib.find("p",class_="star-rating")["class"][-1]] 
        precio = lib.find("p",class_="price_color").text
        stock = lib.find("p",class_="instock availability").text.strip()         # extraigo los detalles de cada libro
        urlLibro = "https://books.toscrape.com/catalogue/" + lib.h3.a["href"]

        pagLibro = scrap.get(urlLibro) # vuelvo a pedir la url + url del libro
        soup2 = BeautifulSoup(pagLibro.text,"html.parser") 
        class_catego = soup2.find("ul",class_="breadcrumb") # busca la categoria clases breadcrumb
        categoria = class_catego.find_all("li")[2].a.text.strip() # busca todas las listas con su href y trae la categoria en text

        # agregro los datos scrapeados en forma de diccionario
        datos_libros.append({
            "titulo":titulo,
            "precio":precio,
            "rating":rating,
            "categoria":categoria,
            "stock":stock,
            "urlLibro":urlLibro,
            "autor":random.choice(autores)
        })
    return datos_libros

todoslosLibros = [] # lista con todos los libros con sus datos
bloquea = threading.Lock() # evita que el hilo agregue los libros a la vez y las agrega uno a uno y mas ordenado

def scrapearPag(pagina):
    resultado = scrapeaTodo(pagina)
    with bloquea:
        todoslosLibros.extend(resultado)

with ThreadPoolExecutor(max_workers=5) as ejecutor: # agregamos hilos para que el scrapeo sea mas rapido
    ejecutor.map(scrapearPag,range(1,51)) # recorre de la pagina 1 al 50

with open("libros.json","w", encoding="utf-8") as fila: # crea un archivo json y lo codifica
    json.dump(todoslosLibros, fila, ensure_ascii=False, indent=4) # agrega los datos al archivo y le de identacion para mas orden

print("scrapeado los 1000 libros")

scrapeado los 1000 libros


In [ ]:
# Inserción de Datos (DML)
# =============================================================================
import psycopg2
import json

import psycopg2
import json

# Leemos el JSON
with open("libros.json", "r", encoding="utf-8") as f:
    todoslosLibros = json.load(f)

# Conectamos a la base de datos
conexionDB = psycopg2.connect(
    host="localhost",
    port="5432",
    database="libros",
    user="postgres",
    password="dan"
)

cursor = conexionDB.cursor()

for libro in todoslosLibros:

    # 1. Insertar autor si no existe y obtener su id
    cursor.execute("SELECT id FROM autores WHERE nombre = %s", (libro["autor"],))
    autor = cursor.fetchone()
    if autor is None:
        cursor.execute("INSERT INTO autores (nombre) VALUES (%s) RETURNING id", (libro["autor"],))
        autor_id = cursor.fetchone()[0]
    else:
        autor_id = autor[0]

    #2. Insertar categoria si no existe y obtener su id
    cursor.execute("SELECT id FROM categorias WHERE nombre = %s",(libro["categoria"],))
    categoria = cursor.fetchone()
    if categoria is None:
        cursor.execute("INSERT INTO categorias (nombre) VALUES (%s) RETURNING id", (libro["categoria"],))
        categoriaID = cursor.fetchone()[0]
    else:
        categoriaID = categoria[0]

        # 3. Insertar libro y obtener su id
    precio_limpio = float(libro["precio"].replace("£", "").strip())
    cursor.execute("""
        INSERT INTO libros (titulo, precio, rating, categoria_id, en_stock, url)
        VALUES (%s, %s, %s, %s, %s, %s) RETURNING id
    """, (libro["titulo"], precio_limpio, libro["rating"], categoriaID, libro["stock"], libro["urlLibro"]))
    libro_id = cursor.fetchone()[0]

    # 4. Insertar en tabla intermedia libros_autores
    cursor.execute("""
        INSERT INTO libros_autores (libro_id, autor_id) VALUES (%s, %s)
    """, (libro_id, autor_id))


conexionDB.commit()
print("libros y autores insertados, para verificar una correcta insercicion")
print("Datos insertados correctamente")
cursor.close()
conexionDB.close()


libros y autores insertados, para verificar una correcta insercicion
Datos insertados correctamente
